<a href="https://colab.research.google.com/github/GLAScorpion/CorsoIA/blob/main/Copy_of_Intelligenza_Artificiale_Lab5_extra_exercises.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## EXERCISE 1: Weather's probability
You are given a (fake) <a href="https://drive.google.com/file/d/1LjZLE9ozaHcBwiCl90mHaS1nXKcglfr4/view">padua_weather.csv</a>
of historical records for Padua's weather. The weather, which can be either rainy (= 1 in the dataset), misty (= 2), or sunny (= 3), is reported for each day of the week, for a whole year (52 weeks).

After you formalised the problem (i.e. identify the random variables and necessary mathematical formulae), write a Python program that reads the dataset and computes the following:
- probability of being sunny during the weekend (one or both days);
- expected weather for each day of the week (*);
- supposed you don't know which day of the week is today: although very unrealistic, how could you guess which day is today based only on the weather?

(\*) An expected value of, for example, 2.5 can be interpreted as "a mix of misty and sunny weather".




In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#load values from csv

import pandas as pd

df = pd.read_csv('drive/MyDrive/padua_weather.csv')

days = df.columns.to_numpy()
print(days)

['Monday' 'Tuesday' 'Wednesday' 'Thursday' 'Friday' 'Saturday' 'Sunday']


In [ ]:
#load data in weather dict

weeks = 52

weather_prob = dict()

for day in days:
  tmp = [0,0,0]
  for x in df[day]:
    tmp[x-1] = tmp[x-1] + 1
  weather_prob[day] = [tmp[0]/weeks,tmp[1]/weeks,tmp[2]/weeks]

print(weather_prob)

{'Monday': [0.3269230769230769, 0.2692307692307692, 0.40384615384615385], 'Tuesday': [0.36538461538461536, 0.28846153846153844, 0.34615384615384615], 'Wednesday': [0.3076923076923077, 0.34615384615384615, 0.34615384615384615], 'Thursday': [0.36538461538461536, 0.3269230769230769, 0.3076923076923077], 'Friday': [0.36538461538461536, 0.3076923076923077, 0.3269230769230769], 'Saturday': [0.38461538461538464, 0.38461538461538464, 0.23076923076923078], 'Sunday': [0.5192307692307693, 0.21153846153846154, 0.2692307692307692]}


In [ ]:
#calc probability of being sunny during (Saturday || Sunday)
rainy = 0
misty = 1
sunny = 2

P_sunny_sat = weather_prob['Saturday'][sunny] * (1 - weather_prob['Sunday'][sunny])
P_sunny_sun = weather_prob['Sunday'][sunny] * (1 - weather_prob['Saturday'][sunny])
P_sunny_both = weather_prob['Saturday'][sunny] * weather_prob['Sunday'][sunny]

P_sunny_wknd = P_sunny_sat + P_sunny_sun + P_sunny_both
print(P_sunny_wknd)

0.4378698224852071


In [ ]:
#calc E[x] for each day
E_x = dict()

for day in days:
  E_x[day] = 0
  cnt = 1
  for x in weather_prob[day]:
    E_x[day] += cnt * x
    cnt+= 1

print(E_x)

{'Monday': 2.076923076923077, 'Tuesday': 1.9807692307692306, 'Wednesday': 2.0384615384615383, 'Thursday': 1.9423076923076923, 'Friday': 1.9615384615384617, 'Saturday': 1.8461538461538463, 'Sunday': 1.75}


In [ ]:
#calc day from weather

#calc prob of each weather type

day_weather = dict()

P_day = 1 / 7

P_w = [0,0,0]

for day in days:
  cnt = 0
  for p in weather_prob[day]:
    P_w[cnt] += p * P_day
    cnt += 1

print(P_w)
#print(P_w[0] + P_w[1] + P_w[2])

#calc list of dict of weather type ranking days by probability

P_d_w = [dict(),dict(),dict()]

for w in range(3):
  tmp_dict = dict()
  for day in days:
    tmp_dict[day] = weather_prob[day][w] * P_day / P_w[w]
  tmp = sorted(tmp_dict, key=tmp_dict.get,reverse=True)
  for day in tmp:
    P_d_w[w][day] = tmp_dict[day]

print(P_d_w[rainy])
print(P_d_w[misty])
print(P_d_w[sunny])

[0.3763736263736263, 0.30494505494505497, 0.31868131868131866]
{'Sunday': 0.19708029197080296, 'Saturday': 0.14598540145985403, 'Tuesday': 0.1386861313868613, 'Thursday': 0.1386861313868613, 'Friday': 0.1386861313868613, 'Monday': 0.12408759124087593, 'Wednesday': 0.11678832116788324}
{'Saturday': 0.18018018018018017, 'Wednesday': 0.16216216216216214, 'Thursday': 0.15315315315315314, 'Friday': 0.14414414414414414, 'Tuesday': 0.1351351351351351, 'Monday': 0.1261261261261261, 'Sunday': 0.09909909909909909}
{'Monday': 0.1810344827586207, 'Tuesday': 0.15517241379310345, 'Wednesday': 0.15517241379310345, 'Friday': 0.14655172413793105, 'Thursday': 0.13793103448275865, 'Sunday': 0.12068965517241378, 'Saturday': 0.10344827586206898}


## EXERCISE 2: Broad Street cholera outbreak

The following is a simplified version of an example in Judea Pearl's *The Book of Why*. It refers to a case of cholera epidemic, caused by contaminated water, which killed hundreds of people in London between 1853 and 1854. The diagram below illustrates some of the key factors explaining this epidemic, in particular:
- $X$ indicates whether the water company's intake was downstream of the London's sewers;
- $W$ indicates whether the water was contaminated or not;
- $Z$ indicates the presence of other external factors (e.g. poverty, miasma, etc.);
- $Y$ indicates the outbreak of cholera.

<img src='https://drive.google.com/uc?id=10O10x_nuuxF55rqRk0TpanHV_7Q819MA'>

(please note the probabilities in the diagram are fake)

> - Formalise the problem using opportune mathematical notations and derive an expression for computing the probability distribution of the cholera given that the water company's intake is upstream (i.e. what is the query? how can it be decomposed?)
> - Write a Python program that computes the actual probabilities of the above distribution using the information from the given CPTs.

In [ ]:
import numpy as np
t,f = 0 , 1

P_z = 0.25
P_x = 0.50

P_W_ZX = np.array([[[0.90, 0.85],[0.10,0.02]],[[0.10,0.15],[0.90,0.98]]])

P_Y_ZW = np.array([[[0.80,0.75],[0.15,0.05]],[[0.20,0.25],[0.85,0.95]]])

print(P_W_ZX[:,:,t])

[[0.9 0.1]
 [0.1 0.9]]


$$
P(Y | ¬x) = P(Y | x) = α P(x) \sum_{z} {\bf P} (z) \sum_{w} {\bf P}(w | z,x) {\bf P}(Y | z,w)
$$

In [ ]:
# prima sommatoria da destra

Phy_w = P_Y_ZW[:,:,t] + P_Y_ZW[:,:,f]

print(Phy_w)

[[1.55 0.2 ]
 [0.45 1.8 ]]
